In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import datetime
from sklearn import *

# load dataset
data_attck = pd.read_csv('datasets/training_data/training_data_attacks.csv') 

/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [28]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

X = data_attck.values[:,:-1]
y = data_attck.values[:,-1]
selector = VarianceThreshold(threshold=0.15)
X_selected = pd.DataFrame(selector.fit_transform(X))
X_selected.columns = data_attck.columns[selector.get_support(indices=True)]
X_selected.head()

,frame_len,sccp_called_digits,sccp_calling_digits,sccp_calling_ssn,sccp_called_ssn,sccp_message_type,tcap_tid,tcap_dtid,tcap_application_context_name,tcap_localValue,...,gsm_map_ss_forwardedToNumber_3,gsm_map_ss_forwardedToNumber_4,gsm_map_ss_ussd_DataCodingScheme,gsm_map_ss_ussd_String,gsm_map_ussd_string,gsm_map_ss_SS_Code_4,gsm_map_ms_apn_Subscribed,gsm_map_ms_apn_InUse,gsm_map_ms_msisdn_2,gsm_map_ms_msisdn_3
0,212.0,4.910192e+11,4.910190e+11,6.0,7.0,17.0,1.112611e+10,1.112611e+10,40010182.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,288.0,4.910192e+11,4.910190e+11,6.0,8.0,17.0,5.110001e+09,7.077430e+07,4001053.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,252.0,4.910192e+11,4.910190e+11,6.0,8.0,17.0,7.077430e+07,7.077430e+07,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,352.0,4.910192e+11,4.910190e+11,6.0,7.0,17.0,0.000000e+00,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,352.0,4.910192e+11,4.910190e+11,6.0,7.0,17.0,0.000000e+00,0.000000e+00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy', verbose=True)
rfecv.fit(X_selected, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [30]:
len(X_selected.columns)

73

In [67]:
clf = ExtraTreesClassifier(n_estimators=20)
clf.fit(X_selected.values[:,60:62], y.astype('int'))

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [56]:
clf = ExtraTreesClassifier(n_estimators=50)
for i in range(60,X_selected.shape[1]):
    print(i)
    clf = clf.fit(X_selected.values[:,:i], y.astype('int'))
    #clf.feature_importances_ 

60


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# debugging the model fittings
feature_importances = pd.DataFrame(model4.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances